In [1]:
!pip install CoolProp

  Using cached CoolProp-6.6.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (7.3 MB)


In [2]:
from CoolProp.CoolProp import PropsSI

A turbojet aircraft flies with a velocity of 260 m/s at an altitude where the air is at 34.5 kPa and -40°C. The compressor has a pressure ratio of 10, and the temperature of the gases at the turbine inlet is 1093°C. Air enters the compressor at a rate of 45 kg/s. Utilizing the cold-air-standard assumptions, determine: 
- (a) the temperature and pressure of the gases at the turbine exit,
- (b) the velocity of the gases at the nozzle exit, and
- (c) the propulsive efficiency of the cycle.

States:
- 1: entering diffuser
- 2: entering compressor
- 3: entering combustion chamber
- 4: entering turbine
- 5: entering nozzle
- 6: exiting nozzle

Assume:
- Steady state

In [15]:
fluid = 'air'

Vaircraft_mPerS = 260
pAir_Pa = 34.5e3
Tair_K = -40+273.15

compressorPressureRatio = 10
# T4_K = 1093 + 273.15
T4_K = 1366.483

mdot_kgPerS = 45

In [16]:
# cold air standard
cp_JperKgperK = PropsSI('C','T',273.15+25,'P',101325,fluid); print(cp_JperKgperK)
k = cp_JperKgperK/PropsSI('O','T',273.15+25,'P',101325,fluid); print(k)

1006.308142514125
1.4017663047263955


#### Process 1 to 2: Isentropic compression of an ideal gas in a diffuser

Assume:
- aircraft is stationary and the air is moving towards the aircraft at a velocity of 260 m/s.
- air exits the diffuser at a negligible velocity (0 m/s).

$$h_2 + w_2^2/2 = h_1 + w_1^2/2 $$

Recall for an ideal gas:

$$h_2 - h_1 = c_p (T_2 - T_1)$$

$$\implies c_p (T_2 - T_1) + w_2^2/2 = w_1^2/2 $$

since the velocity at state 2 is negligible:

$$c_p (T_2 - T_1) = w_1^2/2 $$

$$\implies T_2 = w_1^2/(2c_p) +T_1 $$

In [17]:
w1_mPerS = Vaircraft_mPerS

T1_K = Tair_K

T2_K = (w1_mPerS**2)/(2*cp_JperKgperK)+T1_K; print(T2_K)

266.7381213437817


For an isentropic process, and using our $TdS$ relations, we can write: 

$$p_2 = p_1 \left(\frac{T_2}{T_1} \right)^{k/(k-1)} $$

In [18]:
p1_Pa = pAir_Pa
p2_Pa = p1_Pa*((T2_K/T1_K)**(k/(k-1))); print(p2_Pa)

55176.039169998156


#### Process 2 to 3: Isentropic compression of an ideal gas in a compressor

The compression ratio, $r_p$, is defined as:
$$p_3 = r_p p_2 $$



In [19]:
p3_Pa = compressorPressureRatio*p2_Pa; print(p3_Pa)

551760.3916999815


For an isentropic process, and using our $TdS$ relations, we can write: 

$$T_3 = T_2 \left(\frac{p_3}{p_2} \right)^{(k-1)/k} $$

In [20]:
T3_K = T2_K*(p3_Pa/p2_Pa)**((k-1)/k); print(T3_K)

516.059068504417


#### Process 3 to 4: Combustion chamber

We are only given information about the temperature at the exit of the combustion chamber, as it enters the turbine, $T_4$.

We assume that there is no pressure drop across the chamber: $p_4 = p_3$ 

In [25]:
p4_Pa = p3_Pa

#### Process 4 to 5: Isentropic expansion of an ideal gas in a turbine

Assume: 
- We neglect kinetic energy changes across the compressor and turbine.
- Turbine work is equal to compressor work.

With this we can determine the pressure and temperature at the turbine exit:

$$w_\text{compressor, in} = w_\text{turbine, out}$$

$$\implies h_3 - h_2 = h_4 - h_5$$ 

Again, assuming air behaves as an ideal gas:

$$c_p (T_3 - T_2) = c_p (T_4 - T_5)$$ 

$$\implies T_3 - T_2 = T_4 - T_5$$ 

$$\implies T_5  = T_4 - T_3 + T_2$$ 

In [39]:
T5_K = T4_K - T3_K + T2_K
print('(a) the temperature of the gases at the turbine exit: T5 = {0:1.1f} K.'.format(T5_K))

(a) the temperature of the gases at the turbine exit: T5 = 1117.2 K.


For an isentropic process, and using our $TdS$ relations, we can write: 

$$p_5 = p_4 \left(\frac{T_5}{T_4} \right)^{k/(k-1)} $$

In [29]:
p5_Pa = p4_Pa*(T5_K/T4_K)**(k/(k-1))
print('(a) the pressure of the gases at the turbine exit: p5 = {0:1.1f} Pa.'.format(p5_Pa))

(a) the pressure of the gases at the turbine exit: p5 = 273214.6 Pa.


-------------------

### Part (b)

1) In order to find the air velocity at the nozzle exit, $w_6$, we need to determine the nozzle exit temperature. 

2) Then we can apply the steady-flow energy equation.

#### Process 5-6 Isentropic expansion of an ideal gas in a nozzle

$$T_6 = T_5 \left(\frac{p_6}{p_5} \right)^{(k-1)/k} $$

In [ ]:
p6_Pa = pAir_Pa

T6_K = T5_K*(p6_Pa/p5_Pa)**((k-1)/k); print(T6_K)

Energy analysis of the nozzle:

$$h_6 + w_6^2/2 = h_5 + w_5^2/2 $$

We neglect the velocity of the air entering the nozzle:

$$h_6 + w_6^2/2 = h_5$$

$$\implies w_6 = \sqrt{2(h_5 - h_6)}$$

Assuming ideal gas:

$$\implies w_6 = \sqrt{2 c_p (T_5 - T_6)}$$

In [37]:
w6_mPerS = (2*cp_JperKgperK*(T5_K - T6_K))**(0.5)
print('(b) the air velocity at the nozzle exit is: w6 = {0:1.1f} m/s.'.format(w6_mPerS))

(b) the air velocity at the nozzle exit is: w6 = 1002.9 m/s.


-------------------

### Part (c)

The propulsive efficiency of a turbojet engine is the ratio of the propulsive power developed, $\dot{W}_P$, to the total heat transfer rate to the working fluid:

$$\dot{W}_P = \dot{m} (V_\text{exit} - V_\text{inlet}) V_\text{aircraft} $$


In [34]:
Wdot_W = mdot_kgPerS*(w6_mPerS - w1_mPerS)*Vaircraft_mPerS; print(Wdot_W)

8692491.885722725


The heat transer rate is then:

$$ \dot{Q}_\text{in} = \dot{m} (h_4 - h_3) = \dot{m} c_p (T_4 - T_3)$$

In [36]:
QdotIn_W = mdot_kgPerS*cp_JperKgperK*(T4_K - T3_K); print(QdotIn_W)

38510483.70837958


The efficiency is then: 

$$\eta_P = \frac{\dot{W}_P}{\dot{Q}_\text{in}} $$

In [38]:
eta_P = Wdot_W/QdotIn_W
print('(c) the propulsive efficiency of this cycle is: eta_P = {0:1.1f} %.'.format(eta_P*100))

(c) the propulsive efficiency of this cycle is: eta_P = 22.6 %.
